<a href="https://colab.research.google.com/github/Shahi77/DeepLearning_Assignments/blob/main/Assignment04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Implement the dropout function for a single layer. Draw samples from the uniform distribution S[0,1] . Keep those nodes for which the corresponding sample is greater than p probability , dropping the rest. Implement a dropout_layer function that drops out the elements in the tensor input X with probability dropout, rescaling the remainder. Test the dropout layer with few examples

In [1]:
import torch
import torch.nn.functional as F

def dropout_layer(X: torch.Tensor, dropout: float) -> torch.Tensor:
    """
    Applies dropout to a given input tensor X.

    Parameters:
    X (torch.Tensor): Input tensor.
    dropout (float): Dropout probability (0 <= dropout < 1). Higher means more units dropped.

    Returns:
    torch.Tensor: Tensor after applying dropout (scaled appropriately).
    """
    assert 0 <= dropout < 1, "Dropout probability must be in the range [0, 1)."

    if dropout == 0:
        return X  # No dropout applied

    # Generate a mask with the same shape as X, using uniform distribution
    mask = (torch.rand_like(X) > dropout).float()

    # Scale the remaining elements
    output = (X * mask) / (1.0 - dropout)

    # Debugging info
    print("Original Input:")
    print(X)
    print("Generated Mask:")
    print(mask)
    print("Dropout Applied Output:")
    print(output)
    print(f"Percentage of dropped values: {100 * (1 - mask.mean().item()):.2f}%")
    print(f"Mean of input: {X.mean().item():.4f}, Mean of output: {output.mean().item():.4f}")
    print("-")

    return output

# Testing the dropout layer with a few examples
if __name__ == "__main__":
    torch.manual_seed(42)  # For reproducibility

    # Example input tensor
    X = torch.tensor([[1.0, 2.0, 3.0],
                      [4.0, 5.0, 6.0],
                      [7.0, 8.0, 9.0]])

    dropout_rates = [0.0, 0.3, 0.5, 0.8]

    for p in dropout_rates:
        print(f"Dropout Probability: {p}")
        dropout_layer(X, p)


Dropout Probability: 0.0
Dropout Probability: 0.3
Original Input:
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
Generated Mask:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [0., 1., 1.]])
Dropout Applied Output:
tensor([[ 1.4286,  2.8571,  4.2857],
        [ 5.7143,  7.1429,  8.5714],
        [ 0.0000, 11.4286, 12.8571]])
Percentage of dropped values: 11.11%
Mean of input: 5.0000, Mean of output: 6.0317
-
Dropout Probability: 0.5
Original Input:
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
Generated Mask:
tensor([[0., 1., 1.],
        [1., 1., 1.],
        [0., 1., 1.]])
Dropout Applied Output:
tensor([[ 0.,  4.,  6.],
        [ 8., 10., 12.],
        [ 0., 16., 18.]])
Percentage of dropped values: 22.22%
Mean of input: 5.0000, Mean of output: 8.2222
-
Dropout Probability: 0.8
Original Input:
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
Generated Mask:
tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.

2. Implement Dropout layer in neural network model after every fully connected layer

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.dropout2 = nn.Dropout(p=dropout_prob)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Testing the neural network with dropout
if __name__ == "__main__":
    torch.manual_seed(42)  # For reproducibility

    # Example input tensor
    X = torch.randn(5, 10)  # Batch of 5 samples, 10 features each

    model = NeuralNetwork(input_size=10, hidden_size=20, output_size=5, dropout_prob=0.3)
    model.eval()  # Set model to evaluation mode (no dropout applied)
    print("Output without dropout (evaluation mode):")
    print(model(X))

    model.train()  # Set model to training mode (dropout applied)
    print("Output with dropout (training mode):")
    print(model(X))


Output without dropout (evaluation mode):
tensor([[ 0.0713, -0.0538,  0.0075, -0.3819, -0.2163],
        [-0.0048,  0.1738,  0.2743, -0.2746, -0.1560],
        [ 0.0465,  0.0078,  0.2278, -0.1960, -0.0924],
        [-0.0017,  0.0759,  0.2682, -0.2008, -0.1750],
        [ 0.0606,  0.0299,  0.2935, -0.2621, -0.1218]],
       grad_fn=<AddmmBackward0>)
Output with dropout (training mode):
tensor([[ 0.1122, -0.1865,  0.0692, -0.4563, -0.2607],
        [-0.0957,  0.4482,  0.2956, -0.2055, -0.1704],
        [ 0.0796,  0.1718,  0.2367, -0.2209, -0.1437],
        [-0.0208,  0.2398,  0.3445, -0.2174, -0.1471],
        [ 0.0470,  0.1374,  0.4373, -0.4032, -0.0424]],
       grad_fn=<AddmmBackward0>)


3. Visualization using wandb library for various experimental setups.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb

# Initialize Weights Function
def initialize_weights(m, strategy="random"):
    if isinstance(m, nn.Linear):
        if strategy == "random":
            nn.init.normal_(m.weight, mean=0, std=0.01)
        elif strategy == "pretrained":
            nn.init.xavier_uniform_(m.weight)
        elif strategy == "threshold":
            nn.init.uniform_(m.weight, a=-0.5, b=0.5)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

# Define Standard Neural Network
class StandardNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation, dropout_prob=0.0):
        super(StandardNeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.activation = activation
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define Dropout Neural Network
class DropoutNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation, dropout_prob):
        super(DropoutNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.dropout2 = nn.Dropout(p=dropout_prob)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.activation = activation

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.dropout1(x)
        x = self.activation(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Initialize wandb
wandb.init(project="dropout-experiments")

# Define Experiments
configs = [
    {"model": StandardNeuralNet, "activation": F.logsigmoid, "layers": 2, "units": 100},
    {"model": StandardNeuralNet, "activation": F.logsigmoid, "layers": 2, "units": 800},
    {"model": DropoutNN, "activation": F.logsigmoid, "layers": 3, "units": 1024},
    {"model": DropoutNN, "activation": F.relu, "layers": 3, "units": 1024}
]

# Run Experiments
for config in configs:
    model = config["model"](input_size=784, hidden_size=config["units"], output_size=10, activation=config["activation"], dropout_prob=0.3)
    model.apply(lambda m: initialize_weights(m, strategy="random"))
    wandb.log({"model": str(model), "units": config["units"], "layers": config["layers"]})


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shahi77 (shahi77-national-institute-of-technology-hamirpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
